In [17]:
''''In tis main file we do an initial explortation, we clean the data, and perfoirm EDA and vizualisation'''

import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import initial_exploration as explo
import data_cleaning as cl

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# Load datasets:
cash_requests = pd.read_csv('project_dataset/extract - cash request - data analyst.csv')
fees = pd.read_csv('project_dataset/extract - fees - data analyst - .csv')

In [19]:
cash_requests.head()

,id,amount,status,created_at,updated_at,user_id,moderated_at,deleted_account_id,reimbursement_date,cash_request_received_date,money_back_date,transfer_type,send_at,recovery_status,reco_creation,reco_last_update
0,5,100.0,rejected,2019-12-10 19:05:21.596873+00,2019-12-11 16:47:42.40783+00,804.0,2019-12-11 16:47:42.405646+00,NaN,2020-01-09 19:05:21.596363+00,NaN,NaN,regular,NaN,NaN,NaN,NaN
1,70,100.0,rejected,2019-12-10 19:50:12.34778+00,2019-12-11 14:24:22.900054+00,231.0,2019-12-11 14:24:22.897988+00,NaN,2020-01-09 19:50:12.34778+00,NaN,NaN,regular,NaN,NaN,NaN,NaN
2,7,100.0,rejected,2019-12-10 19:13:35.82546+00,2019-12-11 09:46:59.779773+00,191.0,2019-12-11 09:46:59.777728+00,NaN,2020-01-09 19:13:35.825041+00,NaN,NaN,regular,NaN,NaN,NaN,NaN
3,10,99.0,rejected,2019-12-10 19:16:10.880172+00,2019-12-18 14:26:18.136163+00,761.0,2019-12-18 14:26:18.128407+00,NaN,2020-01-09 19:16:10.879606+00,NaN,NaN,regular,NaN,NaN,NaN,NaN
4,1594,100.0,rejected,2020-05-06 09:59:38.877376+00,2020-05-07 09:21:55.34008+00,7686.0,2020-05-07 09:21:55.320193+00,NaN,2020-06-05 22:00:00+00,NaN,NaN,regular,NaN,NaN,NaN,NaN


In [20]:
# Inital exploration:
explo.check(cash_requests)

Number of columns: 16 amd rows: 23970

Data types:
id                              int64
amount                        float64
status                         object
created_at                     object
updated_at                     object
user_id                       float64
moderated_at                   object
deleted_account_id            float64
reimbursement_date             object
cash_request_received_date     object
money_back_date                object
transfer_type                  object
send_at                        object
recovery_status                object
reco_creation                  object
reco_last_update               object
dtype: object

Unique values count:
id                            23970
amount                           41
status                            7
created_at                    23970
updated_at                    23970
user_id                       10798
moderated_at                  16035
deleted_account_id             1141
reimbursement_dat

In [21]:
cl.convert_dates(cash_requests, cl.cash_request_date_columns)
cash_requests.head()

,id,amount,status,created_at,updated_at,user_id,moderated_at,deleted_account_id,reimbursement_date,cash_request_received_date,money_back_date,transfer_type,send_at,recovery_status,reco_creation,reco_last_update
0,5,100.0,rejected,10/12/2019,11/12/2019,804.0,11/12/2019,NaN,09/01/2020,NaN,NaN,regular,NaN,NaN,NaN,NaN
1,70,100.0,rejected,10/12/2019,11/12/2019,231.0,11/12/2019,NaN,09/01/2020,NaN,NaN,regular,NaN,NaN,NaN,NaN
2,7,100.0,rejected,10/12/2019,11/12/2019,191.0,11/12/2019,NaN,09/01/2020,NaN,NaN,regular,NaN,NaN,NaN,NaN
3,10,99.0,rejected,10/12/2019,18/12/2019,761.0,18/12/2019,NaN,09/01/2020,NaN,NaN,regular,NaN,NaN,NaN,NaN
4,1594,100.0,rejected,06/05/2020,07/05/2020,7686.0,07/05/2020,NaN,NaN,NaN,NaN,regular,NaN,NaN,NaN,NaN


In [22]:
fees.head()

,id,cash_request_id,type,status,category,total_amount,reason,created_at,updated_at,paid_at,from_date,to_date,charge_moment
0,6537,14941.0,instant_payment,rejected,NaN,5.0,Instant Payment Cash Request 14941,2020-09-07 10:47:27.42315+00,2020-10-13 14:25:09.396112+00,2020-12-17 14:50:07.47011+00,NaN,NaN,after
1,6961,11714.0,incident,accepted,rejected_direct_debit,5.0,rejected direct debit,2020-09-09 20:51:17.998653+00,2020-10-13 14:25:15.537063+00,2020-12-08 17:13:10.45908+00,NaN,NaN,after
2,16296,23371.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 23371,2020-10-23 10:10:58.352972+00,2020-10-23 10:10:58.352994+00,2020-11-04 19:34:37.43291+00,NaN,NaN,after
3,20775,26772.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 26772,2020-10-31 15:46:53.643958+00,2020-10-31 15:46:53.643982+00,2020-11-19 05:09:22.500223+00,NaN,NaN,after
4,11242,19350.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 19350,2020-10-06 08:20:17.170432+00,2020-10-13 14:25:03.267983+00,2020-11-02 14:45:20.355598+00,NaN,NaN,after


In [23]:
explo.check(fees)

Number of columns: 13 amd rows: 21061

Data types:
id                   int64
cash_request_id    float64
type                object
status              object
category            object
total_amount       float64
reason              object
created_at          object
updated_at          object
paid_at             object
from_date           object
to_date             object
charge_moment       object
dtype: object

Unique values count:
id                 21061
cash_request_id    12933
type                   3
status                 4
category               2
total_amount           2
reason             15149
created_at         21026
updated_at         21061
paid_at            15529
from_date           1084
to_date             1560
charge_moment          2
dtype: int64

This columns apear to be categroical:
 Index(['type', 'status', 'category', 'total_amount', 'charge_moment'], dtype='object')

Unique value count for categorical columns:
type
instant_payment    11099
postpone            77

In [24]:
cl.convert_dates(fees, cl.fees_data_date_columns)
fees.head()

,id,cash_request_id,type,status,category,total_amount,reason,created_at,updated_at,paid_at,from_date,to_date,charge_moment
0,6537,14941.0,instant_payment,rejected,NaN,5.0,Instant Payment Cash Request 14941,07/09/2020,13/10/2020,17/12/2020,NaN,NaN,NaN
1,6961,11714.0,incident,accepted,rejected_direct_debit,5.0,rejected direct debit,09/09/2020,13/10/2020,08/12/2020,NaN,NaN,NaN
2,16296,23371.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 23371,23/10/2020,23/10/2020,04/11/2020,NaN,NaN,NaN
3,20775,26772.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 26772,31/10/2020,31/10/2020,19/11/2020,NaN,NaN,NaN
4,11242,19350.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 19350,06/10/2020,13/10/2020,02/11/2020,NaN,NaN,NaN


In [25]:
data_df = cl.merge_df(cash_requests, fees, 'outer','id', 'cash_request_id')
data_df.head()

,id_x,amount,status_x,created_at_x,updated_at_x,user_id,moderated_at,deleted_account_id,reimbursement_date,cash_request_received_date,money_back_date,transfer_type,send_at,recovery_status,reco_creation,reco_last_update,id_y,cash_request_id,type,status_y,category,total_amount,reason,created_at_y,updated_at_y,paid_at,from_date,to_date,charge_moment
0,5.0,100.0,rejected,10/12/2019,11/12/2019,804.0,11/12/2019,NaN,09/01/2020,NaN,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,70.0,100.0,rejected,10/12/2019,11/12/2019,231.0,11/12/2019,NaN,09/01/2020,NaN,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7.0,100.0,rejected,10/12/2019,11/12/2019,191.0,11/12/2019,NaN,09/01/2020,NaN,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10.0,99.0,rejected,10/12/2019,18/12/2019,761.0,18/12/2019,NaN,09/01/2020,NaN,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1594.0,100.0,rejected,06/05/2020,07/05/2020,7686.0,07/05/2020,NaN,NaN,NaN,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
explo.check(data_df)

Number of columns: 29 amd rows: 32098

Data types:
id_x                          float64
amount                        float64
status_x                       object
created_at_x                   object
updated_at_x                   object
user_id                       float64
moderated_at                   object
deleted_account_id            float64
reimbursement_date             object
cash_request_received_date     object
money_back_date                object
transfer_type                  object
send_at                        object
recovery_status                object
reco_creation                  object
reco_last_update               object
id_y                          float64
cash_request_id               float64
type                           object
status_y                       object
category                       object
total_amount                  float64
reason                         object
created_at_y                   object
updated_at_y                   object

In [27]:
data_df = cl.remove_nan(data_df, "id_x")
explo.check_null(data_df)

4 rows were removed

Count of null values:
id_x                              0
amount                            0
status_x                          0
created_at_x                      0
updated_at_x                      0
user_id                        2572
moderated_at                  10564
deleted_account_id            29521
reimbursement_date            28033
cash_request_received_date     7945
money_back_date               14890
transfer_type                     0
send_at                        9724
recovery_status               24894
reco_creation                 24894
reco_last_update              24894
id_y                          11037
cash_request_id               11037
type                          11037
status_y                      11037
category                      29898
total_amount                  11037
reason                        11037
created_at_y                  11037
updated_at_y                  11037
paid_at                       16656
from_date            

In [28]:
frames = cl.selecting_data_types(data_df)
num = frames[1]
cat = frames[0]

In [29]:
num.head()

,id_x,amount,user_id,deleted_account_id,id_y,cash_request_id
0,5.0,100.0,804.0,NaN,NaN,NaN
1,70.0,100.0,231.0,NaN,NaN,NaN
2,7.0,100.0,191.0,NaN,NaN,NaN
3,10.0,99.0,761.0,NaN,NaN,NaN
4,1594.0,100.0,7686.0,NaN,NaN,NaN


In [30]:
cat.head()

,status_x,created_at_x,updated_at_x,moderated_at,reimbursement_date,cash_request_received_date,money_back_date,transfer_type,send_at,recovery_status,reco_creation,reco_last_update,type,status_y,category,reason,created_at_y,updated_at_y,paid_at,from_date,to_date,charge_moment,total_amount
0,rejected,10/12/2019,11/12/2019,11/12/2019,09/01/2020,NaN,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rejected,10/12/2019,11/12/2019,11/12/2019,09/01/2020,NaN,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rejected,10/12/2019,11/12/2019,11/12/2019,09/01/2020,NaN,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rejected,10/12/2019,18/12/2019,18/12/2019,09/01/2020,NaN,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rejected,06/05/2020,07/05/2020,07/05/2020,NaN,NaN,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:

# then we can selecet numerical and categorical columns , and divide the columns between us and ecah work in a set of columns performing an 
# univariate analysis (frequency plots, box plots, doughnut, histograms   or any other you concider)
#For numercial plots also create report in mean , meadian, std, and arrive to conclusions 

In [32]:
# Ensure correct data types again:
cl.ensure_correct_data_types(cash_requests, cl.cash_request_date_columns)

# Check data types:
print("\nCheck data types:")
print(cash_requests[['created_at', 'updated_at', 'reimbursement_date', 
                     'cash_request_received_date', 'money_back_date']].dtypes)

# Check NaT:
print("\nCheck NaT:")
print(cash_requests[['created_at', 'updated_at', 'reimbursement_date', 
                     'cash_request_received_date', 'money_back_date']].isna().sum())



Check data types:
created_at                    datetime64[ns]
updated_at                    datetime64[ns]
reimbursement_date            datetime64[ns]
cash_request_received_date    datetime64[ns]
money_back_date               datetime64[ns]
dtype: object

Check NaT:
created_at                        0
updated_at                        0
reimbursement_date            20920
cash_request_received_date     7681
money_back_date               11930
dtype: int64
